# Demos: Lecture 18

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

from lecture18_helpers import *

## Demo 1: mitigating error in noisy VQE simulations

Ground state of $H_2$ looks like:

$$
|\psi_g(\theta)\rangle = \cos(\theta/2) |1100\rangle - \sin(\theta/2) |0011\rangle
$$

In [ ]:
bond_length = 1.3228
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -bond_length/2, 0.0, 0.0, bond_length/2])

H, n_qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)

In [ ]:
dev = qml.device("default.qubit", wires=4)

def prepare_ground_state(theta):
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    qml.DoubleExcitation(theta, wires=range(4))
    return qml.expval(H)

optimal_theta = np.array(0.20973288, requires_grad=True)

In [ ]:
from qiskit.test.mock import FakeSantiago
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel

In [ ]:
device = QasmSimulator.from_backend(FakeSantiago())

noise_model = NoiseModel.from_backend(device, readout_error=False)

noisy_dev = qml.device(
    "qiskit.aer", backend='qasm_simulator', wires=4, shots=10000, noise_model=noise_model
)

Explore what happens with a single term in the Hamiltonian.

## Demo 2: simple quantum state tomography

In [ ]:
mixed_dev = qml.device("default.mixed", wires=4)

@qml.qnode(mixed_dev)
def prepare_h_state(theta):
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    qml.DoubleExcitation(theta, wires=range(4))
    return qml.state()

In [ ]:
ideal_state = prepare_h_state(optimal_theta)

## Demo 3: Tomography x ZNE